REsizing

In [2]:
from CellClicker.cell_clicker import ImageViewer
import tkinter as tk
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageViewer(root)
    root.mainloop()


In [20]:
from CellClicker.image_selector_multiphase import load_ui_from_folder

load_ui_from_folder()

Sara
Selected XML File: E:/Scott/Data/20240417\user_selections\Sara.xml
Selected XML: E:/Scott/Data/20240417\user_selections\Sara.xml
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t058.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t058.png
E:\Scott\Data\20240417\images\20240417_P1_t060.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t058.png
E:\Scott\Data\20240417\images\20240417_P1_t060.png
E:\Scott\Data\20240417\images\20240417_P1_t054.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png

In [18]:
import os
import xml.etree.ElementTree as ET

def parse_xml_for_phases(xml_file):
    """ Parse XML and get phase indices for each image and series. """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    image_data = {}
    for entry in root.findall('DataEntry'):
        path = entry.find('PathName').text
        series_id = entry.find('SeriesID').text
        indices = {
            phase.tag: int(phase.text) for phase in entry
            if phase.tag in ['prometaphase', 'metaphase', 'anaphase'] and phase.text.isdigit()
        }
        image_data[(path, series_id)] = indices
    return image_data

def parse_xml_for_labels(xml_file):
    """ Parse annotation XML and return structured label data. """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    labels_data = {}
    for path_element in root.findall('path'):
        image_path = path_element.find('name').text
        for series in path_element.findall('series'):
            series_id = series.get('id')
            labels = []
            for label in series.findall('label'):
                label_info = {
                    'class_id': int(label.find('class_id').text),
                    'x_center': float(label.find('x_center').text),
                    'y_center': float(label.find('y_center').text),
                    'width': float(label.find('width').text),
                    'height': float(label.find('height').text)
                }
                labels.append(label_info)
            labels_data[(image_path, series_id)] = labels
    return labels_data

def generate_filename(base_path, offset):
    """ Generate a new filename based on an offset from the base filename """
    base_name = os.path.basename(base_path)
    prefix, number, ext = base_name.rsplit('_', 2)
    new_number = int(number[1:]) + offset
    new_filename = f"{prefix}_t{str(new_number).zfill(3)}.{ext}"
    return new_filename

def create_yolo_labels(phase_data, labels_data, output_dir):
    """ Generate YOLO label files considering class_id as filename reference. """
    os.makedirs(output_dir, exist_ok=True)
    # this gets from cell_reigons the path and series id, and all the indicies
    for (path, series_id), indices in labels_data.items():
        print(path)
        phases = phase_data.get((path, series_id), [])
        print(phases)
        if phases:
            
            for label in indices:
                

                new_index = len(indices) - int(label['class_id'])
                print(f"newindex {new_index}")
                print(f"looping {label}")
                # Calculate new filename based on class_id which is used as an offset here
                # label = phases[index]


                # print(label['class_id'])
                # print(- label['class_id'] + index)
                print(path)
                
                new_filename = generate_filename(path, new_index )
                print(new_filename)
                
                # yolo_label = f"{label['class_id']} {label['x_center']} {label['y_center']} {label['width']} {label['height']}"
                
                # # Write label to corresponding file
                # filename = os.path.join(output_dir, new_filename.replace('.png', '.txt'))
                # with open(filename, 'a') as file:
                #     file.write(yolo_label + '\n')

# Usage
phase_data = parse_xml_for_phases('E:/Scott/Data/20240417/user_selections/Sara.xml')
labels_data = parse_xml_for_labels('E:/Scott/Data/20240417/images/cell_reigons.xml')
create_yolo_labels(phase_data, labels_data, 'E:/Scott/Data/20240417/new_labels')


E:/Scott/Data/20240417/images/20240417_P1_t047.png
{'prometaphase': 4, 'metaphase': 9, 'anaphase': 15}
newindex 16
looping {'class_id': 0, 'x_center': 0.0484375, 'y_center': 0.9070833333333334, 'width': 0.028125, 'height': 0.05416666666666667}
E:/Scott/Data/20240417/images/20240417_P1_t047.png
20240417_t017.t047.png
newindex 15
looping {'class_id': 1, 'x_center': 0.04739583333333333, 'y_center': 0.9029166666666667, 'width': 0.028125, 'height': 0.05416666666666667}
E:/Scott/Data/20240417/images/20240417_P1_t047.png
20240417_t016.t047.png
newindex 14
looping {'class_id': 2, 'x_center': 0.04791666666666667, 'y_center': 0.89875, 'width': 0.028125, 'height': 0.05416666666666667}
E:/Scott/Data/20240417/images/20240417_P1_t047.png
20240417_t015.t047.png
newindex 13
looping {'class_id': 3, 'x_center': 0.0484375, 'y_center': 0.8945833333333333, 'width': 0.028125, 'height': 0.05416666666666667}
E:/Scott/Data/20240417/images/20240417_P1_t047.png
20240417_t014.t047.png
newindex 12
looping {'class_

In [5]:
phase_data

{('E:/Scott/Data/20240417/images/20240417_P1_t047.png',
  '1'): {'prometaphase': 4, 'metaphase': 9, 'anaphase': 15},
 ('E:/Scott/Data/20240417/images/20240417_P1_t014.png', '1'): {},
 ('E:/Scott/Data/20240417/images/20240417_P1_t058.png',
  '1'): {'prometaphase': 4, 'metaphase': 8, 'anaphase': 13},
 ('E:/Scott/Data/20240417/images/20240417_P1_t060.png',
  '1'): {'prometaphase': 5, 'metaphase': 8},
 ('E:/Scott/Data/20240417/images/20240417_P1_t060.png',
  '2'): {'prometaphase': 8, 'metaphase': 16},
 ('E:/Scott/Data/20240417/images/20240417_P1_t054.png',
  '1'): {'prometaphase': 4, 'metaphase': 9, 'anaphase': 14},
 ('E:/Scott/Data/20240417/images/20240417_P2_t025.png', '1'): {}}

In [6]:
labels_data

{('E:/Scott/Data/20240417/images/20240417_P1_t047.png',
  '1'): [{'class_id': 0,
   'x_center': 0.0484375,
   'y_center': 0.9070833333333334,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 1,
   'x_center': 0.04739583333333333,
   'y_center': 0.9029166666666667,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 2,
   'x_center': 0.04791666666666667,
   'y_center': 0.89875,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 3,
   'x_center': 0.0484375,
   'y_center': 0.8945833333333333,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 4,
   'x_center': 0.04895833333333333,
   'y_center': 0.8904166666666666,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 5,
   'x_center': 0.049479166666666664,
   'y_center': 0.88875,
   'width': 0.028125,
   'height': 0.05416666666666667}, {'class_id': 6,
   'x_center': 0.05,
   'y_center': 0.8854166666666666,
   'width': 0.028125,
   'height': 0.054